# Load packages and libraries

In [1]:
.libPaths()
.libPaths(new = c("/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"))
.libPaths()
# sessionInfo()
Sys.setenv(RETICULATE_PYTHON="/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/bin/python")

suppressMessages(library(Seurat))
suppressMessages(library(Signac))
 suppressMessages(library(EnsDb.Hsapiens.v86)) 
suppressMessages(library(dplyr)) 
suppressMessages(library(ggplot2)) 
suppressMessages(library(Matrix)) 
suppressMessages(library(data.table)) 
suppressMessages(library(ggpubr)) 
suppressMessages(library(ggplot2))
suppressMessages(library(pheatmap))
suppressMessages(library(presto))
suppressMessages(library("qlcMatrix"))
suppressMessages(library("cowplot"))
suppressMessages(library("RColorBrewer"))
suppressMessages(library("plyr"))
suppressMessages(library("forcats"))
suppressMessages(library('ggeasy'))
suppressMessages(library('dplyr'))
suppressMessages(library("svglite"))
suppressMessages(library("ape"))
suppressMessages(library("ggforce"))
suppressMessages(library("tidyr"))
suppressMessages(library("edgeR"))
suppressMessages(library("apeglm"))
suppressMessages(library("DESeq2"))
suppressMessages(library("tibble"))

suppressMessages(library(BSgenome.Hsapiens.NCBI.GRCh38))
suppressMessages(library(BSgenome.Hsapiens.UCSC.hg38))


[1] "/home/manuel.tardaguila/R/x86_64-conda-linux-gnu-library/4.3"
[2] "/home/manuel.tardaguila/conda_envs/multiome_QC/lib/R/library"

[1] "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"
[2] "/home/manuel.tardaguila/conda_envs/multiome_QC/lib/R/library"

Warning message:
“package ‘Seurat’ was built under R version 4.3.2”
Warning message:
“package ‘SeuratObject’ was built under R version 4.3.3”
Warning message:
“package ‘Signac’ was built under R version 4.3.2”
Warning message:
“package ‘BiocGenerics’ was built under R version 4.3.2”
Warning message:
“package ‘GenomicRanges’ was built under R version 4.3.2”
Warning message:
“package ‘S4Vectors’ was built under R version 4.3.2”
Warning message:
“package ‘IRanges’ was built under R version 4.3.2”
Warning message:
“package ‘GenomeInfoDb’ was built under R version 4.3.2”
Warning message:
“package ‘Biobase’ was built under R version 4.3.2”
Warning message:
"package 'Matrix' was built under R version 4.3.3"
Warning message:
"package 'qlcMatrix' was built under R version 4.3.3"
Warning message:
"package 'slam' was built under R version 4.3.3"
Warning message:
"package 'sparsesvd' was built under R version 4.3.3"
Warning message:
"package 'ggeasy' was built under R version 4.3.3"
Warning messag

In [2]:
#sessionInfo()

# Checkpoint after integrative genotyping

In [3]:
setwd("/group/soranzo/manuel.tardaguila/2025_hESC_MK_multiome/processing_outputs/")

adata<-readRDS(file="merged_processed_presentation_updated_integrative_genotyping.rds")

adata

An object of class Seurat 
445320 features across 37504 samples within 4 assays 
Active assay: SCT (31512 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 3 other assays present: RNA, RNA_raw, ATAC
 7 dimensional reductions calculated: pca, umap.rna, lsi, umap.atac, umap.wnn, harmony.rna, harmony.atac

In [4]:
#colnames(adata@meta.data)

# Create a genotype factor

In [5]:

summary(adata@meta.data$Assigned_GFPbc_integral)

adata@meta.data$Assigned_GFPgenotype_integral<-NA

adata@meta.data$Assigned_GFPgenotype_integral[which(adata@meta.data$Assigned_GFPbc_integral%in%c('chrGFP_WTA','chrGFP_WTB','chrGFP_WTC'))]<-'wt'
adata@meta.data$Assigned_GFPgenotype_integral[which(adata@meta.data$Assigned_GFPbc_integral%in%c('chrGFP_rs1','chrGFP_rs2','chrGFP_rs3'))]<-'CHEK2 T/T'
adata@meta.data$Assigned_GFPgenotype_integral[which(adata@meta.data$Assigned_GFPbc_integral%in%c('chrGFP_R882H1','chrGFP_R882H2','chrGFP_R882H3'))]<-'DNMT3A R882H'
adata@meta.data$Assigned_GFPgenotype_integral[which(adata@meta.data$Assigned_GFPbc_integral%in%c('chrGFP_rs_R882H1','chrGFP_rs_R882H2','chrGFP_rs_R882H3'))]<-'Double mutants'


summary(as.factor(adata@meta.data$Assigned_GFPgenotype_integral))


adata@meta.data$Assigned_GFPgenotype_integral<-factor(adata@meta.data$Assigned_GFPgenotype_integral,
                                                     levels=c('wt','CHEK2 T/T','DNMT3A R882H','Double mutants'),
                                                     ordered=T)


summary(adata@meta.data$Assigned_GFPgenotype_integral)



chrGFP_WTA       chrGFP_WTB       chrGFP_WTC       chrGFP_rs1 
             130              226              305              872 
      chrGFP_rs2       chrGFP_rs3    chrGFP_R882H1    chrGFP_R882H2 
             776             7996             1101              123 
   chrGFP_R882H3 chrGFP_rs_R882H1 chrGFP_rs_R882H2 chrGFP_rs_R882H3 
            2355             1381              706              864 
         Doublet         Negative 
           10781             9888

CHEK2 T/T   DNMT3A R882H Double mutants             wt           NA's 
          9644           3579           2951            661          20669

wt      CHEK2 T/T   DNMT3A R882H Double mutants           NA's 
           661           9644           3579           2951          20669

# Subset my object for cells with genotype assigned

In [6]:
adata_geno<-subset(adata, Assigned_GFPgenotype_integral != "NA")

adata_geno

Warning message:
"Removing 20669 cells missing data for vars requested"


An object of class Seurat 
445320 features across 16835 samples within 4 assays 
Active assay: SCT (31512 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 3 other assays present: RNA, RNA_raw, ATAC
 7 dimensional reductions calculated: pca, umap.rna, lsi, umap.atac, umap.wnn, harmony.rna, harmony.atac

# Create a variable to correct for clone line

In [7]:
 adata_geno$sample_id<-adata_geno$Assigned_GFPbc_integral
  
  cat(sprintf(as.character(names(summary(adata_geno$sample_id)))))
  cat("\n")
  cat(sprintf(as.character(summary(adata_geno$sample_id))))
  cat("\n")

chrGFP_WTA chrGFP_WTB chrGFP_WTC chrGFP_rs1 chrGFP_rs2 chrGFP_rs3 chrGFP_R882H1 chrGFP_R882H2 chrGFP_R882H3 chrGFP_rs_R882H1 chrGFP_rs_R882H2 chrGFP_rs_R882H3 Doublet Negative
130 226 305 872 776 7996 1101 123 2355 1381 706 864 0 0


# Read DE genes

In [8]:
setwd("/group/soranzo/manuel.tardaguila/2025_hESC_MK_multiome/Downstream_analysis/")

Global_results<-as.data.frame(read.table(file="DE_results_without_time_as_a_covariate_NEW_METHOD.tsv", sep="\t", header=T), stringsAsFactors=F)

str(Global_results)
cat("\n")

Global_normalized<-as.data.frame(read.table(file="Normalized_gene_counts_NEW_METHOD.tsv", sep="\t", header=T), stringsAsFactors=F)

str(Global_normalized)
cat("\n")

SIG<-as.data.frame(read.table(file="DE_results_without_time_as_a_covariate_NEW_METHOD_only_padj_less_0.05.tsv", sep="\t", header=T), stringsAsFactors=F)

str(SIG)
cat("\n")

'data.frame':	658818 obs. of  8 variables:
 $ gene          : chr  "DACH1" "CYTIP" "TANC1" "ROBO1" ...
 $ baseMean      : num  292.8 29.9 92.8 97.7 111.2 ...
 $ log2FoldChange: num  -0.60416 -0.00375 -0.79762 -0.81306 -0.00554 ...
 $ lfcSE         : num  0.1771 0.0248 0.2968 0.2864 0.0253 ...
 $ pvalue        : num  2.25e-05 1.59e-04 1.79e-04 1.74e-04 8.20e-05 ...
 $ padj          : num  0.665 0.753 0.753 0.753 0.753 ...
 $ contrast      : chr  "Genotype_Double.mutants_vs_wt" "Genotype_Double.mutants_vs_wt" "Genotype_Double.mutants_vs_wt" "Genotype_Double.mutants_vs_wt" ...
 $ cell_type     : chr  "MK" "MK" "MK" "MK" ...

'data.frame':	2598671 obs. of  4 variables:
 $ Symbol        : chr  "MIR1302-2HG" "FAM138A" "OR4F5" "AL627309.1" ...
 $ count         : num  0 0 0 3.5 0 ...
 $ GFPbc_integral: chr  "chrGFP_R882H1" "chrGFP_R882H1" "chrGFP_R882H1" "chrGFP_R882H1" ...
 $ current_anot  : chr  "MK" "MK" "MK" "MK" ...

'data.frame':	780 obs. of  8 variables:
 $ gene          : chr  "PCDH11X

In [9]:
adata_geno

An object of class Seurat 
445320 features across 16835 samples within 4 assays 
Active assay: SCT (31512 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 3 other assays present: RNA, RNA_raw, ATAC
 7 dimensional reductions calculated: pca, umap.rna, lsi, umap.atac, umap.wnn, harmony.rna, harmony.atac

In [10]:
DefaultAssay(adata_geno)<-'ATAC'

In [50]:
peaks_PRE <- GetAssayData(adata_geno,layer="counts")

head(peaks_PRE)

  [[ suppressing 34 column names 'MCO_01326_AAACAGCCACAGCCAT-1', 'MCO_01326_AAACAGCCAGGAAGCC-1', 'MCO_01326_AAACAGCCAGTATGTT-1' ... ]]



6 x 16835 sparse Matrix of class "dgCMatrix"
                                                                              
chr1-10076-10321   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1-180715-181606 . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2 .
chr1-191225-191765 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1-629567-630655 1 1 1 1 . 3 . 4 4 . 1 2 2 . . 1 2 1 1 3 1 . 1 2 . . . 1 . 1
chr1-631349-631875 . . . . . 1 . . . . . . . . . . . . . . . . . . . . . . . .
chr1-632832-633078 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
                                 
chr1-10076-10321   . . . . ......
chr1-180715-181606 . . . . ......
chr1-191225-191765 . . . . ......
chr1-629567-630655 . 2 1 1 ......
chr1-631349-631875 . . . . ......
chr1-632832-633078 . . . . ......

 .....suppressing 16801 columns in show(); maybe adjust 'options(max.print= *, width = *)'
 ..............................

# TF IDF normalize ATAC data

In [51]:
RunTFIDF(adata_geno[['ATAC']])


Performing TF-IDF normalization



ChromatinAssay data with 340606 features for 16835 cells
Variable features: 340606 
Genome: hg38 
Annotation present: TRUE 
Motifs present: FALSE 
Fragment files: 1 

In [52]:
peaks_POST <- GetAssayData(adata_geno,layer="counts")

head(peaks_POST)

  [[ suppressing 34 column names 'MCO_01326_AAACAGCCACAGCCAT-1', 'MCO_01326_AAACAGCCAGGAAGCC-1', 'MCO_01326_AAACAGCCAGTATGTT-1' ... ]]



6 x 16835 sparse Matrix of class "dgCMatrix"
                                                                              
chr1-10076-10321   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1-180715-181606 . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2 .
chr1-191225-191765 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1-629567-630655 1 1 1 1 . 3 . 4 4 . 1 2 2 . . 1 2 1 1 3 1 . 1 2 . . . 1 . 1
chr1-631349-631875 . . . . . 1 . . . . . . . . . . . . . . . . . . . . . . . .
chr1-632832-633078 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
                                 
chr1-10076-10321   . . . . ......
chr1-180715-181606 . . . . ......
chr1-191225-191765 . . . . ......
chr1-629567-630655 . 2 1 1 ......
chr1-631349-631875 . . . . ......
chr1-632832-633078 . . . . ......

 .....suppressing 16801 columns in show(); maybe adjust 'options(max.print= *, width = *)'
 ..............................

In [53]:
identical(peaks_PRE,peaks_POST)

[1] TRUE

The data was already normalized

# Link Peaks to DE genes

In [17]:
# $ bash ~/Scripts/Wraper_scripts/133_Link_peaks.sh /group/soranzo/manuel.tardaguila/2025_hESC_MK_multiome/ Downstream_analysis

# Perform DA

# Subset for Linked Peaks

In [18]:
setwd("/group/soranzo/manuel.tardaguila/2025_hESC_MK_multiome/Downstream_analysis/")

In [19]:
LinkedPeaks<-readRDS(file="Linked_peak_to_selected_genes.rds")

In [20]:
str(LinkedPeaks)

'data.frame':	8746 obs. of  11 variables:
 $ seqnames     : Factor w/ 24 levels "chr1","chr2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ start        : int  5717468 5764659 5766687 5831502 5897165 5949489 6141768 6148525 6183751 6205209 ...
 $ end          : int  6209389 6209389 6209389 6209389 6209389 6209389 6209389 6209389 6209389 6209389 ...
 $ width        : int  491922 444731 442703 377888 312225 259901 67622 60865 25639 4181 ...
 $ strand       : Factor w/ 3 levels "+","-","*": 3 3 3 3 3 3 3 3 3 3 ...
 $ score        : num  0.1025 0.0549 0.1003 0.13 0.1072 ...
 $ Symbol       : chr  "RPL22" "RPL22" "RPL22" "RPL22" ...
 $ Peak_ID      : chr  "chr1-5716645-5718290" "chr1-5764546-5764772" "chr1-5766017-5767357" "chr1-5830392-5832611" ...
 $ zscore       : num  2.69 2.3 1.77 2.56 2.14 ...
 $ pvalue       : num  0.00362 0.01064 0.03877 0.00523 0.01614 ...
 $ Minus_logpval: num  2.44 1.97 1.41 2.28 1.79 4.08 1.64 1.53 2.04 1.88 ...


## Deine peak_subset assay

In [25]:
adata_geno[["peak_subset"]] <- CreateChromatinAssay(
    counts = peak_subset,
    fragments = Fragments(adata_geno),
    min.cells = 0, min.features = -1 )
  
  DefaultAssay(adata_geno) <- 'peak_subset'

## Define interest regions

In [26]:
### GRCh38 rs62237617

rs_vector<-c("rs62237617")
VAR_vector<-c("chr22_28365160_C_T")

selected_variants_df<-as.data.frame(cbind(rs_vector,VAR_vector))
  
  colnames(selected_variants_df)<-c('rs','VAR')
  
  selected_variants_df$chr<-gsub("_.+$","",selected_variants_df$VAR)
  selected_variants_df$pos38<-gsub("^[^_]+_","",selected_variants_df$VAR)
  selected_variants_df$pos38<-as.integer(gsub("_.+$","",selected_variants_df$pos38))
  selected_variants_df$ref<-gsub("^[^_]+_[^_]+_","",selected_variants_df$VAR)
  selected_variants_df$ref<-gsub("_.+$","",selected_variants_df$ref)
  selected_variants_df$alt<-gsub("^[^_]+_[^_]+_[^_]+_","",selected_variants_df$VAR)
  
  cat("selected_variants_df_0\n")
  cat(str(selected_variants_df))
  cat("\n")

selected_variants_df_0
'data.frame':	1 obs. of  6 variables:
 $ rs   : chr "rs62237617"
 $ VAR  : chr "chr22_28365160_C_T"
 $ chr  : chr "chr22"
 $ pos38: int 28365160
 $ ref  : chr "C"
 $ alt  : chr "T"



In [27]:
 gr_VARS <- GRanges(
    seqnames = as.character(gsub("chr","",selected_variants_df$chr)),
    name2=as.character(selected_variants_df$rs),
    ranges=IRanges(
      start=as.numeric(selected_variants_df$pos38),
      end=as.numeric(selected_variants_df$pos38),
      name=selected_variants_df$VAR))
  
  cat("gr_VARS_0\n")
  cat(str(gr_VARS))
  cat("\n")

gr_VARS_0
Formal class 'GRanges' [package "GenomicRanges"] with 7 slots
  ..@ seqnames       :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. ..@ values         : Factor w/ 1 level "22": 1
  .. .. ..@ lengths        : int 1
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ ranges         :Formal class 'IRanges' [package "IRanges"] with 6 slots
  .. .. ..@ start          : int 28365160
  .. .. ..@ width          : int 1
  .. .. ..@ NAMES          : chr "chr22_28365160_C_T"
  .. .. ..@ elementType    : chr "ANY"
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ strand         :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. ..@ values         : Factor w/ 3 levels "+","-","*": 3
  .. .. ..@ lengths        : int 1
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ seqinfo        :Formal class 'Seqinfo' [package "GenomeInfoDb"] with 4 slots
  .. .. ..@ seqnames   : chr "22"
  .. .. ..@ s

In [28]:
  gr_region <- GRanges(
    seqnames = as.character("22"),
    name2=as.character("ROI_1"),
    ranges=IRanges(
      start=as.numeric(28360000),
      end=as.numeric(28380000),
      name='ROI_1'))
  
  cat("gr_region\n")
  cat(str(gr_region))
  cat("\n")

gr_region
Formal class 'GRanges' [package "GenomicRanges"] with 7 slots
  ..@ seqnames       :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. ..@ values         : Factor w/ 1 level "22": 1
  .. .. ..@ lengths        : int 1
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ ranges         :Formal class 'IRanges' [package "IRanges"] with 6 slots
  .. .. ..@ start          : int 28360000
  .. .. ..@ width          : int 20001
  .. .. ..@ NAMES          : chr "ROI_1"
  .. .. ..@ elementType    : chr "ANY"
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ strand         :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. ..@ values         : Factor w/ 3 levels "+","-","*": 3
  .. .. ..@ lengths        : int 1
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ seqinfo        :Formal class 'Seqinfo' [package "GenomeInfoDb"] with 4 slots
  .. .. ..@ seqnames   : chr "22"
  .. .. ..@ seqlengths

In [29]:
gr_VARS<-append(gr_VARS, gr_region)

## Find the overlap

In [30]:
 #### Intersect Peak to genes with SNP ----
  
  DEBUG <- 1
  
  m <- findOverlaps(gr_VARS,gr_peaks)
  
  if(DEBUG == 1)
  {
    cat("m\n")
    cat(str(m))
    cat("\n")
  }
  
  subjectHits_peaks<-subjectHits(m)
  
  if(DEBUG == 1)
  {
    cat("subjectHits_peaks\n")
    cat(str(subjectHits_peaks))
    cat("\n")
  }
  
  queryHits_VARS<-queryHits(m)
  
  if(DEBUG == 1)
  {
    cat("queryHits_VARS\n")
    cat(str(queryHits_VARS))
    cat("\n")
  }
  
  VARS_df <- data.frame(rs=as.character(gr_VARS$name2),
                        VAR=names(gr_VARS), stringsAsFactors = F)
  
  if(DEBUG == 1)
  {
    cat("VARS_df_0\n")
    cat(str(VARS_df))
    cat("\n")
  }
  
  VARS_df_hits<-VARS_df[queryHits_VARS,]
  
  if(DEBUG == 1)
  {
    cat("VARS_df_hits_0\n")
    cat(str(VARS_df_hits))
    cat("\n")
  }
  
  peaks_df_SNP <- data.frame(chr=as.character(seqnames(gr_peaks)),
                                     start=as.integer(start(gr_peaks)),
                                     end=as.integer(end(gr_peaks)),
                                     Peak_ID=names(gr_peaks),
                                     stringsAsFactors = F)
  
  
  if(DEBUG == 1)
  {
    cat("peaks_df_SNP_0\n")
    cat(str(peaks_df_SNP))
    cat("\n")
  }
  
  peaks_df_SNP_hits<-peaks_df_SNP[subjectHits_peaks,]
  
  if(dim(peaks_df_SNP_hits)[1] >0)
  {
    if(DEBUG == 1)
    {
      cat("peaks_df_SNP_hits_0\n")
      cat(str(peaks_df_SNP_hits))
      cat("\n")
    }
    
    peaks_df_SNP_hits<-cbind(peaks_df_SNP_hits,VARS_df_hits)
    
    peaks_df_SNP_hits$chr<-paste('chr',peaks_df_SNP_hits$chr,sep='')
    
    if(DEBUG == 1)
    {
      cat("peaks_df_SNP_hits_1\n")
      cat(str(peaks_df_SNP_hits))
      cat("\n")
    }
    
 
       
  }#dim(peaks_df_SNP_hits)[1] >0

m
Formal class 'SortedByQueryHits' [package "S4Vectors"] with 6 slots
  ..@ from           : int [1:4] 2 2 2 2
  ..@ to             : int [1:4] 195865 195866 195867 195868
  ..@ nLnode         : int 2
  ..@ nRnode         : int 340606
  ..@ elementMetadata: NULL
  ..@ metadata       : list()

subjectHits_peaks
 int [1:4] 195865 195866 195867 195868

queryHits_VARS
 int [1:4] 2 2 2 2

VARS_df_0
'data.frame':	2 obs. of  2 variables:
 $ rs : chr  "rs62237617" "ROI_1"
 $ VAR: chr  "chr22_28365160_C_T" "ROI_1"

VARS_df_hits_0
'data.frame':	4 obs. of  2 variables:
 $ rs : chr  "ROI_1" "ROI_1" "ROI_1" "ROI_1"
 $ VAR: chr  "ROI_1" "ROI_1" "ROI_1" "ROI_1"

peaks_df_SNP_0
'data.frame':	340606 obs. of  4 variables:
 $ chr    : chr  "1" "1" "1" "1" ...
 $ start  : int  10076 180715 191225 629567 631349 632832 633491 777545 779621 817115 ...
 $ end    : int  10321 181606 191765 630655 631875 633078 634520 779326 779846 817388 ...
 $ Peak_ID: chr  "chr1-10076-10321" "chr1-180715-181606" "chr1-191225

In [34]:
adata_geno

An object of class Seurat 
445324 features across 16835 samples within 5 assays 
Active assay: peak_subset (4 features, 0 variable features)
 2 layers present: counts, data
 4 other assays present: RNA, RNA_raw, SCT, ATAC
 7 dimensional reductions calculated: pca, umap.rna, lsi, umap.atac, umap.wnn, harmony.rna, harmony.atac

## Extract peak_subset counts

In [26]:
 matrix_ATAC<-GetAssayData(object = adata_geno, assay = "peak_subset", layer = "counts")
  
  cat("matrix_ATAC\n")
  cat(str(matrix_ATAC))
  cat("\n")

matrix_ATAC
Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:8912623] 0 46 50 58 76 87 96 124 170 205 ...
  ..@ p       : int [1:16836] 0 285 857 1539 2773 3799 4960 5707 6984 7289 ...
  ..@ Dim     : int [1:2] 8086 16835
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:8086] "chr1-5716645-5718290" "chr1-5764546-5764772" "chr1-5766017-5767357" "chr1-5830392-5832611" ...
  .. ..$ : chr [1:16835] "MCO_01326_AAACAGCCACAGCCAT-1" "MCO_01326_AAACAGCCAGGAAGCC-1" "MCO_01326_AAACAGCCAGTATGTT-1" "MCO_01326_AAACATGCAAACGCGA-1" ...
  ..@ x       : num [1:8912623] 1 1 1 1 1 1 1 1 1 1 ...
  ..@ factors : list()



## Extract metadata

In [27]:
  metadata<-droplevels(adata_geno[[]])

cat(sprintf(as.character(names(summary(metadata$time_point)))))
  cat("\n")
  cat(sprintf(as.character(summary(metadata$time_point))))
  cat("\n")
  
  
  cat(sprintf(as.character(names(summary(metadata$current_anot)))))
  cat("\n")
  cat(sprintf(as.character(summary(metadata$current_anot))))
  cat("\n")
  
  
  cat(sprintf(as.character(names(summary(as.factor(metadata$Assigned_GFPbc_integral))))))
  cat("\n")
  cat(sprintf(as.character(summary(as.factor(metadata$Assigned_GFPbc_integral)))))
  cat("\n")
  
  cat(sprintf(as.character(names(summary(as.factor(metadata$sample_id))))))
  cat("\n")
  cat(sprintf(as.character(summary(as.factor(metadata$sample_id)))))
  cat("\n")


Day_0 Day_8 Day_12 Day_15
5852 259 4905 5819
hESC HemogenicEndothelium MEP early erythroid early MK MK
5705 191 3532 678 466 6263
chrGFP_WTA chrGFP_WTB chrGFP_WTC chrGFP_rs1 chrGFP_rs2 chrGFP_rs3 chrGFP_R882H1 chrGFP_R882H2 chrGFP_R882H3 chrGFP_rs_R882H1 chrGFP_rs_R882H2 chrGFP_rs_R882H3
130 226 305 872 776 7996 1101 123 2355 1381 706 864
chrGFP_WTA chrGFP_WTB chrGFP_WTC chrGFP_rs1 chrGFP_rs2 chrGFP_rs3 chrGFP_R882H1 chrGFP_R882H2 chrGFP_R882H3 chrGFP_rs_R882H1 chrGFP_rs_R882H2 chrGFP_rs_R882H3
130 226 305 872 776 7996 1101 123 2355 1381 706 864


## Create a new Seurat object with the ATAC and the metadata

In [28]:

  ATAC_object <- CreateSeuratObject(counts = matrix_ATAC, assay = "RNA",
                                   meta.data=metadata)

ATAC_object

An object of class Seurat 
8086 features across 16835 samples within 1 assay 
Active assay: RNA (8086 features, 0 variable features)
 1 layer present: counts

## Aggregate by sample_id (clone line) and current_anot (cell type)

In [29]:
 cell_type_names <- levels(metadata[,which(colnames(metadata) == 'current_anot')])
  
  cat("cell_type_names_0\n")
  cat(str(cell_type_names))
  cat("\n")
  
  sample_names <- levels(metadata[,which(colnames(metadata) == 'sample_id')])
  
  cat("sample_names_0\n")
  cat(str(sample_names))
  cat("\n")
  
  groups <- metadata[,c(which(colnames(metadata) == 'sample_id'),which(colnames(metadata) == 'current_anot'))]
  
  cat("groups_0\n")
  cat(str(groups))
  cat("\n")
  
  aggr_counts <- Seurat2PB(ATAC_object, sample="sample_id", cluster="current_anot")
  
  cat("aggr_counts_0\n")
  cat(str(aggr_counts))
  cat("\n")

cell_type_names_0
 chr [1:6] "hESC" "HemogenicEndothelium" "MEP" "early erythroid" "early MK" ...

sample_names_0
 chr [1:12] "chrGFP_WTA" "chrGFP_WTB" "chrGFP_WTC" "chrGFP_rs1" ...

groups_0
'data.frame':	16835 obs. of  2 variables:
 $ sample_id   : Factor w/ 12 levels "chrGFP_WTA","chrGFP_WTB",..: 9 4 7 7 7 11 7 7 9 9 ...
 $ current_anot: Ord.factor w/ 6 levels "hESC"<"HemogenicEndothelium"<..: 1 1 1 1 1 1 1 1 1 1 ...

aggr_counts_0
Formal class 'DGEList' [package "edgeR"] with 1 slot
  ..@ .Data:List of 3
  .. ..$ : num [1:8086, 1:71] 0 0 0 0 0 0 0 4 0 1 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:8086] "chr1-5716645-5718290" "chr1-5764546-5764772" "chr1-5766017-5767357" "chr1-5830392-5832611" ...
  .. .. .. ..$ : chr [1:71] "chrGFP_R882H1_clusterearly erythroid" "chrGFP_R882H1_clusterearly MK" "chrGFP_R882H1_clusterHemogenicEndothelium" "chrGFP_R882H1_clusterhESC" ...
  .. ..$ :'data.frame':	71 obs. of  5 variables:
  .. .. ..$ group       : Factor w/ 1 le

## Go to a list format

In [30]:
 ## Initiate empty list
  counts_ls <- list()
  
  DEBUG<-0
  

for (i in 1:length(cell_type_names)) {
    
    cell_type_names[i]
    
    ## Extract indexes of columns in the global matrix that match a given cluster
    column_idx <- which(tstrsplit(colnames(aggr_counts), "_cluster")[[2]] == cell_type_names[i])
    
    sub_aggr<- aggr_counts[, column_idx]
    
    if(DEBUG == 1)
    {
      cat("sub_aggr_0\n")
      cat(str(sub_aggr))
      cat("\n")
      
    }
    
    ## Store corresponding sub-matrix as one element of a list
    counts_ls[[i]] <-sub_aggr
    names(counts_ls)[i] <- cell_type_names[i]

    #break
    
  }

In [31]:
 # Explore the different components of the list
  
 cat("counts_ls_0\n")
# cat(str(counts_ls))
    cat("\n")

# Each index of the list is a cell type with the counts in all the sample_ids (time_point + clone_line)

counts_ls_0



In [33]:
#counts_ls[['MK']]$counts

## Create a metadata list

In [35]:
#colnames(metadata)

In [36]:
 #### Create group level variables -----
  
  # Extract sample-level variables
  metadata_NEW <- metadata %>% 
    as.data.frame() %>% 
    dplyr::select(Assigned_GFPgenotype_integral, Assigned_GFPbc_integral, sample_id)
  
cat("metadata_NEW_0\n")
  cat(str(metadata_NEW))
  cat("\n")
  
  # Exclude duplicated rows
  metadata_NEW <- metadata_NEW[!duplicated(metadata_NEW), ]


cat("metadata_NEW_0.5\n")
  cat(str(metadata_NEW))
  cat("\n")
  
 
  # Rename rows
  rownames(metadata_NEW) <- metadata_NEW$sample_id
  
  cat("metadata_NEW_1\n")
  cat(str(metadata_NEW))
  cat("\n")
  

metadata_NEW_0
'data.frame':	16835 obs. of  3 variables:
 $ Assigned_GFPgenotype_integral: Ord.factor w/ 4 levels "wt"<"CHEK2 T/T"<..: 3 2 3 3 3 4 3 3 3 3 ...
 $ Assigned_GFPbc_integral      : Ord.factor w/ 12 levels "chrGFP_WTA"<"chrGFP_WTB"<..: 9 4 7 7 7 11 7 7 9 9 ...
 $ sample_id                    : Factor w/ 12 levels "chrGFP_WTA","chrGFP_WTB",..: 9 4 7 7 7 11 7 7 9 9 ...

metadata_NEW_0.5
'data.frame':	12 obs. of  3 variables:
 $ Assigned_GFPgenotype_integral: Ord.factor w/ 4 levels "wt"<"CHEK2 T/T"<..: 3 2 3 4 4 1 2 4 3 2 ...
 $ Assigned_GFPbc_integral      : Ord.factor w/ 12 levels "chrGFP_WTA"<"chrGFP_WTB"<..: 9 4 7 11 12 3 6 10 8 5 ...
 $ sample_id                    : Factor w/ 12 levels "chrGFP_WTA","chrGFP_WTB",..: 9 4 7 11 12 3 6 10 8 5 ...

metadata_NEW_1
'data.frame':	12 obs. of  3 variables:
 $ Assigned_GFPgenotype_integral: Ord.factor w/ 4 levels "wt"<"CHEK2 T/T"<..: 3 2 3 4 4 1 2 4 3 2 ...
 $ Assigned_GFPbc_integral      : Ord.factor w/ 12 levels "chrGFP_WTA"<"chrGF

In [37]:
t <- table(metadata$sample_id,
             metadata$current_anot)
  
  cat("t_0\n")
  cat(str(t))
  cat("\n")

t_0
 'table' int [1:12, 1:6] 59 133 76 127 113 252 875 93 2199 496 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:12] "chrGFP_WTA" "chrGFP_WTB" "chrGFP_WTC" "chrGFP_rs1" ...
  ..$ : chr [1:6] "hESC" "HemogenicEndothelium" "MEP" "early erythroid" ...



In [38]:
##### Creating metadata list ----------------
  
  ## Initiate empty list
  metadata_ls <- list()
  
  DEBUG<-0
  
  for (i in 1:length(counts_ls)) {
    
    ## Initiate a data frame for cluster i with one row per sample (matching column names in the counts matrix)
    df <- data.frame(current_anot_sample_id = colnames(counts_ls[[i]]))

      if(DEBUG == 1){

         cat("df_0\n")
         cat(str(df))
         cat("\n")
      }
    
    ## Use tstrsplit() to separate cluster (cell type) and sample IDs
    df$current_anot_id <- tstrsplit(df$current_anot_sample_id, "_cluster")[[2]]
    df$sample_id  <- tstrsplit(df$current_anot_sample_id, "_cluster")[[1]]
    
    
     if(DEBUG == 1){

         cat("df_1\n")
         cat(str(df))
         cat("\n")
      }
    
    
    ## Retrieve cell count information for this cluster from global cell count table
    idx <- which(colnames(t) == unique(df$current_anot_id))

       if(DEBUG == 1){

                 cat("idx_0\n")
                 cat(str(idx))
                 cat("\n")          
           }
      
    cell_counts <- t[, idx]

       if(DEBUG == 1){

                 cat("cell_counts_0\n")
                 cat(str(cell_counts))
                 cat("\n")          
           }
 
    
    
    
    
    
    ## Remove samples with zero cell contributing to the cluster
    cell_counts <- cell_counts[cell_counts > 0]
    
    if(DEBUG == 1){

                 cat("cell_counts_1\n")
                 cat(str(cell_counts))
                 cat("\n")          
           }
    
    
    ## Match order of cell_counts and sample_ids
    sample_order <- match(df$sample_id, names(cell_counts))
    cell_counts <- cell_counts[sample_order]

      if(DEBUG == 1){

                 cat("cell_counts_2\n")
                 cat(str(cell_counts))
                 cat("\n")          
           }
    
    
    
    ## Append cell_counts to data frame
    df$cell_count <- cell_counts

       if(DEBUG == 1){

         cat("df_2\n")
         cat(str(df))
         cat("\n")
      }
    
    
    ## Join data frame (capturing metadata specific to cluster) to generic metadata
    df <- plyr::join(df, metadata_NEW, 
                     by = intersect(names(df), names(metadata_NEW)))

        if(DEBUG == 1){

         cat("df_3\n")
         cat(str(df))
         cat("\n")
      }
    
    ## Update rownames of metadata to match colnames of count matrix, as needed later for DE
    rownames(df) <- df$current_anot_sample_id

         if(DEBUG == 1){

         cat("df_4\n")
         cat(str(df))
         cat("\n")
      }
    
    ## Store complete metadata for cluster i in list
    metadata_ls[[i]] <- df
    names(metadata_ls)[i] <- unique(df$cluster_id)

     # break
    
  }

In [67]:
 # Explore the different components of the list
  
  #cat("metadata_ls_0\n")
  #cat(str(metadata_ls))
  #cat("\n")

# List of df. Each index is a cell type. Each df per index has the combination of current anot (cell type) and sample_id (time_point and clone line), the cell count of each of this triple combination together with the accesory column of GFPbc, GFPgenotype and time_point

# DESeq2 for DA analysis

In [39]:
# Double-check that both lists have same names
  check_1<-all(names(counts_ls) == names(metadata_ls))
  
  cat("check_1\n")
  cat(sprintf(as.character(check_1)))
  cat("\n")

check_1
TRUE


In [40]:
array_cell_annotations<-names(counts_ls)

str(array_cell_annotations)

 chr [1:6] "hESC" "HemogenicEndothelium" "MEP" "early erythroid" "early MK" ...


### Loop within a cell type

In [44]:
DEBUG<-0

Global_results<-data.frame()

Global_normalized<-data.frame()

for(i in 1:length(array_cell_annotations)){

    Results_per_comparison_within_cell_type<-data.frame()


    cell_type_sel<-array_cell_annotations[i]

    cat("----------------------------------->\t")
    cat(sprintf(as.character(cell_type_sel)))
    cat("\n")

    idx.sel<-which(names(counts_ls) == cell_type_sel)
      cluster_counts <- counts_ls[[idx.sel]]

    if(DEBUG == 1){
          cat("cluster_counts_0\n")
          cat(str(cluster_counts))
          cat("\n")
    }

     cell_type_metadata <- metadata_ls[[idx.sel]]

      if(DEBUG == 1){
          cat("cell_type_metadata_0\n")
          cat(str(cell_type_metadata))
          cat("\n")
    }

    
      cell_type_metadata$Genotype<-as.character(cell_type_metadata$Assigned_GFPgenotype_integral)
      cell_type_metadata$Genotype<-factor(cell_type_metadata$Genotype)
      cell_type_metadata$Genotype<-relevel(cell_type_metadata$Genotype, ref='wt') ### Nothing works with ordered factors


       if(DEBUG == 1){
          cat("cell_type_metadata_1\n")
          cat(str(cell_type_metadata))
          cat("\n")
          cat(str(unique(cell_type_metadata$current_anot_id)))
          cat("\n")
    }

    check_2<-all(colnames(cluster_counts) == rownames(cell_type_metadata))

    if(DEBUG == 1){
      cat("check_2\n")
      cat(sprintf(as.character(check_2)))
      cat("\n")
        cat(sprintf(as.character(rownames(cell_type_metadata))))
      cat("\n")
    }


    
    ### DESeq data set with all the columns the contrast will be every column against the reference wt level set before

     dds_NEW <- DESeqDataSetFromMatrix(cluster_counts, 
                                  colData = cell_type_metadata, 
                                  design = ~ Genotype)

    ### Normalize ----------------------------------

    dds_NEW<-estimateSizeFactors(dds_NEW)

    #### Run DESeq LRT comparing to the reduced model

    dds_NEW_lrt <- DESeq(dds_NEW, test = "LRT", reduced = ~ 1)

   
     ###### Obtain all the possible contrasts --------------------------

    possible_contrasts<-colnames(dds_NEW_lrt@modelMatrix)[-1] # -1 because 1 is the Intercept term

    for(iteration_contrasts in 1:length(possible_contrasts)){

        contrast_sel<-possible_contrasts[iteration_contrasts]

        cat("------------------------------------->\t")
        cat(sprintf(as.character(contrast_sel)))
        cat("\n")
        

        ############ obtain the result for this contrast with its specific pvalue coming from Wald test see ?results

        tmp_results<-results(dds_NEW_lrt, test= "Wald", name=contrast_sel)

        #### expand LogFC #########################################
    
        tmp_results <- lfcShrink(dds_NEW_lrt, 
                                     coef = contrast_sel,
                                     res=tmp_results,
                                     type = "apeglm")

          if(DEBUG == 1){
          cat("tmp_results_0\n")
          cat(str(tmp_results))
          cat("\n")
            }


          #### obtain data frame #########################################

          tmp_tb <- as.data.frame(tmp_results %>%
                      data.frame() %>%
                      rownames_to_column(var = "Peak_ID") %>%
                      as_tibble() %>%
                      arrange(padj), stringsAsFactors=F)
                    
                    

        if(DEBUG == 1){
          cat("tmp_tb_0\n")
          cat(str(tmp_tb))
          cat("\n")
            }

        tmp_tb$contrast<-contrast_sel


        if(DEBUG == 1){
          cat("tmp_tb_1\n")
          cat(str(tmp_tb))
          cat("\n")
            }


        Results_per_comparison_within_cell_type<-rbind(tmp_tb,Results_per_comparison_within_cell_type)
    

        

        }#iteration_contrasts in 1:length(possible_contrasts)


    Results_per_comparison_within_cell_type$cell_type<-cell_type_sel

    Global_results<-rbind(Results_per_comparison_within_cell_type,Global_results)

    #### Extract normalized counts to plot heatmaps later ----------------------------------------

    nor_counts<-as.data.frame(counts(dds_NEW, normalized=TRUE))

    nor_counts$Peak_ID<-row.names(nor_counts)
    row.names(nor_counts)<-NULL

    nor_counts.m<-melt(nor_counts, id.vars='Peak_ID', variable.name="sample_id_current_annot_id", value.name="count")
    nor_counts.m$GFPbc_integral<-gsub("_cluster.+$","",nor_counts.m$sample_id_current_annot_id)
    nor_counts.m$current_anot<-gsub("^.+_cluster","",nor_counts.m$sample_id_current_annot_id)    
    nor_counts.m<-nor_counts.m[,-which(colnames(nor_counts.m) == 'sample_id_current_annot_id')]

    if(DEBUG == 1){
       cat("nor_counts.m_0\n")
         str(nor_counts.m)
         cat("\n")
        }
    

    Global_normalized<-rbind(nor_counts.m, Global_normalized)

    
    
}# i in 1:length(array_cell_annotations

----------------------------------->	hESC


converting counts to integer mode

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

  No

------------------------------------->	Genotype_CHEK2.T.T_vs_wt


using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

Warning message in nbinomGLM(x = x, Y = YNZ, size = size, weights = weightsNZ, offset = offsetNZ, :
"the line search routine failed, unable to sufficiently decrease the function value"


------------------------------------->	Genotype_DNMT3A.R882H_vs_wt


...

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

Warning message in nbinomGLM(x = x, Y = YNZ, size = size, weights = weightsNZ, offset = offsetNZ, :
"the line search routine failed, unable to sufficiently decrease the function value"
Warning message in nbinomGLM(x = x, Y = YNZ, size = size, weights = weightsNZ, offset = offsetNZ, :
"the line search routine failed, unable to sufficiently decrease the function value"
Warning message in nbinomGLM(x = x, Y = YNZ, size = size, weights = weightsNZ, offset = offsetNZ, :
"the line search routine failed, unable to sufficiently decrease the function value"
Warning message in nbinomGLM(x = x, Y = YNZ, size = size, weights = weightsNZ, offset = offsetNZ, :
"the line search routine failed, unable

------------------------------------->	Genotype_DNMT3A.R882H_vs_wt


using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



------------------------------------->	Genotype_Double.mutants_vs_wt


using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

Warning message in melt(nor_counts, id.vars = "Peak_ID", variable.name = "sample_id_current_annot_id", :
"The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(nor_counts). In the next version, this warning will become an error."


In [45]:
str(Global_results)

'data.frame':	145548 obs. of  8 variables:
 $ Peak_ID       : chr  "chr1-5716645-5718290" "chr1-5764546-5764772" "chr1-5766017-5767357" "chr1-5830392-5832611" ...
 $ baseMean      : num  0.967 0.119 0.526 1.718 0.171 ...
 $ log2FoldChange: num  4.92e-07 -7.88e-08 2.17e-07 -1.10e-06 7.82e-08 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.579 0.744 0.852 0.429 0.989 ...
 $ padj          : num  1 1 1 1 1 ...
 $ contrast      : chr  "Genotype_Double.mutants_vs_wt" "Genotype_Double.mutants_vs_wt" "Genotype_Double.mutants_vs_wt" "Genotype_Double.mutants_vs_wt" ...
 $ cell_type     : chr  "MK" "MK" "MK" "MK" ...


In [48]:
str(Global_normalized)

'data.frame':	574106 obs. of  4 variables:
 $ Peak_ID       : chr  "chr1-5716645-5718290" "chr1-5764546-5764772" "chr1-5766017-5767357" "chr1-5830392-5832611" ...
 $ count         : num  3.69 0 0 4.92 0 ...
 $ GFPbc_integral: chr  "chrGFP_R882H1" "chrGFP_R882H1" "chrGFP_R882H1" "chrGFP_R882H1" ...
 $ current_anot  : chr  "MK" "MK" "MK" "MK" ...


In [55]:
colnames(LinkedPeaks)[which(colnames(LinkedPeaks) == 'pvalue')]<-'pvalue_Links'
colnames(LinkedPeaks)[which(colnames(LinkedPeaks) == 'Minus_logpval')]<-'Minus_logpval_Links'

In [56]:
Global_results<-merge(Global_results,  LinkedPeaks, by ='Peak_ID')

In [57]:
str(Global_results)

'data.frame':	157428 obs. of  18 variables:
 $ Peak_ID            : chr  "chr1-102834710-102835233" "chr1-102834710-102835233" "chr1-102834710-102835233" "chr1-102834710-102835233" ...
 $ baseMean           : num  0.4392 0.0681 0.1381 0.0404 0.049 ...
 $ log2FoldChange     : num  -2.17e-08 5.97e-08 5.44e-03 -3.77e-09 -1.12e-08 ...
 $ lfcSE              : num  0.00144 0.00144 0.27509 0.00144 0.00144 ...
 $ pvalue             : num  0.992 0.606 0.776 0.907 0.833 ...
 $ padj               : num  0.998 1 0.918 0.952 1 ...
 $ contrast           : chr  "Genotype_CHEK2.T.T_vs_wt" "Genotype_CHEK2.T.T_vs_wt" "Genotype_CHEK2.T.T_vs_wt" "Genotype_DNMT3A.R882H_vs_wt" ...
 $ cell_type          : chr  "HemogenicEndothelium" "MK" "early MK" "early erythroid" ...
 $ seqnames           : Factor w/ 24 levels "chr1","chr2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ start              : int  102834972 102834972 102834972 102834972 102834972 102834972 102834972 102834972 102834972 102834972 ...
 $ end                :

In [59]:
SIG<-Global_results[which(Global_results$padj < 0.05),]

str(SIG)

'data.frame':	0 obs. of  18 variables:
 $ Peak_ID            : chr 
 $ baseMean           : num 
 $ log2FoldChange     : num 
 $ lfcSE              : num 
 $ pvalue             : num 
 $ padj               : num 
 $ contrast           : chr 
 $ cell_type          : chr 
 $ seqnames           : Factor w/ 24 levels "chr1","chr2",..: 
 $ start              : int 
 $ end                : int 
 $ width              : int 
 $ strand             : Factor w/ 3 levels "+","-","*": 
 $ score              : num 
 $ Symbol             : chr 
 $ zscore             : num 
 $ pvalue_Links       : num 
 $ Minus_logpval_Links: num 


In [60]:
summary(as.factor(SIG$contrast))

integer(0)

# SAVE DA results

In [61]:
setwd("/group/soranzo/manuel.tardaguila/2025_hESC_MK_multiome/Downstream_analysis/")

write.table(Global_results, file="DA_results_without_time_as_a_covariate_NEW_METHOD_only_linked_peaks_to_DE_genes.tsv", sep="\t", quote=F, row.names=F)

In [62]:
setwd("/group/soranzo/manuel.tardaguila/2025_hESC_MK_multiome/Downstream_analysis/")

write.table(Global_normalized, file="Normalized_Peak_counts_NEW_METHOD_only_linked_peaks_to_DE_genes.tsv", sep="\t", quote=F, row.names=F)

In [64]:
str(adata_geno@meta.data$current_anot)

 Ord.factor w/ 6 levels "hESC"<"HemogenicEndothelium"<..: 1 1 1 1 1 1 1 1 1 1 ...


In [65]:
summary(adata_geno@meta.data$current_anot[which(as.numeric(adata_geno@meta.data$current_anot) ==1)])

hESC HemogenicEndothelium                  MEP 
                5705                    0                    0 
     early erythroid             early MK                   MK 
                   0                    0                    0

In [70]:
ct<-as.character(unique(adata_geno@meta.data$current_anot[which(as.numeric(adata_geno@meta.data$current_anot) ==4)]))

ct

[1] "early erythroid"

In [73]:
adata_geno_cell_type_sel<-subset(adata_geno, current_anot == ct)

In [74]:
adata_geno_cell_type_sel

An object of class Seurat 
453406 features across 678 samples within 5 assays 
Active assay: peak_subset (8086 features, 0 variable features)
 2 layers present: counts, data
 4 other assays present: RNA, RNA_raw, SCT, ATAC
 7 dimensional reductions calculated: pca, umap.rna, lsi, umap.atac, umap.wnn, harmony.rna, harmony.atac